# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os


# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,15.69,70,Clouds,6.14,ZA,1727913535
1,1,stanley,54.8680,-1.6985,6.40,95,Clear,1.54,GB,1727913537
2,2,papatowai,-46.5619,169.4708,8.43,98,Rain,6.76,NZ,1727913538
3,3,lebu,-37.6167,-73.6500,9.24,81,Clouds,9.51,CL,1727913539
4,4,ballina,-28.8667,153.5667,18.27,63,Clouds,6.11,AU,1727913540


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
city_data_df.hvplot.points?

Signature:
city_data_df.hvplot.points(
    x=None,
    y=None,
    *,
    alpha,
    angle,
    cmap,
    color,
    fill_alpha,
    fill_color,
    hit_dilation,
    hover_alpha,
    hover_color,
    hover_fill_alpha,
    hover_fill_color,
    hover_line_alpha,
    hover_line_cap,
    hover_line_color,
    hover_line_dash,
    hover_line_dash_offset,
    hover_line_join,
    hover_line_width,
    line_alpha,
    line_cap,
    line_color,
    line_dash,
    line_dash_offset,
    line_join,
    line_width,
    marker,
    muted,
    muted_alpha,
    muted_color,
    muted_fill_alpha,
    muted_fill_color,
    muted_line_alpha,
    muted_line_cap,
    muted_line_color,
    muted_line_dash,
    muted_line_dash_offset,
    muted_line_join,
    muted_line_width,
    nonselection_alpha,
    nonselection_color,
    nonselection_fill_alpha,
    nonselection_fill_color,
    nonselection_line_alpha,
    nonselection_line_cap,
    nonselection_line_color,
    nonselection_line_dash,
    nonselect

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True,  
    size='humidity', 
    color='red',  
    title='CT Map',  
    frame_width=800,  
    frame_height=600,  
    tiles=True,
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
Maxtemp=city_data_df['Max Temp']
Maxtemp.head()

0    15.69
1     6.40
2     8.43
3     9.24
4    18.27
Name: Max Temp, dtype: float64

In [43]:
# A max temperature lower than 27 degrees but higher than 21

# Wind speed less than 4.5 m/s

# Zero cloudiness

In [50]:
Ideal_cities=city_data_df[(city_data_df['Max Temp']>19)&(city_data_df['Max Temp']<26)&(city_data_df['Wind Speed']<4.5)&(city_data_df['Cloudiness']=="Clear")].dropna()

### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# list/array colums
hotel_df=Ideal_cities.copy()
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,wake village,33.4268,-94.1063,23.36,60,Clear,2.55,US,1727913645
141,141,gebeit,21.0667,36.3167,24.85,44,Clear,2.49,SD,1727913706
162,162,ha'il,27.5219,41.6907,25.01,31,Clear,1.03,SA,1727913729
170,170,belmonte,-15.8631,-38.8828,23.04,82,Clear,4.16,BR,1727913740
173,173,sabha,27.0377,14.4283,24.85,30,Clear,3.25,LY,1727913743


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000  #
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 1,
    # "filter": f"circle:{longitude},{latitude},{radius}", 
    # "bias": f"proximity:{longitude},{latitude}"

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    print(row)
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"] 
    longitude=row["Lng"] 

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # name_address = # YOUR CODE HERE
    response=requests.get(base_url, params=params)
    print(response.url)
    # Convert the API response to JSON format
    name_address = response.json()# YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
City_ID                                        87
City                                 wake village
Lat                                       33.4268
Lng                                      -94.1063
Max Temp                                    23.36
Humidity                                       60
Cloudiness                                  Clear
Wind Speed                                   2.55
Country                                        US
Date                                   1727913645
Hotel Name    Country Inn & Suites, Texarkana, TX
Name: 87, dtype: object
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=cd14440c5acb4e48becb628ec51a9b3d&limit=1&filter=circle%3A-94.1063%2C33.4268%2C10000&bias=proximity%3A-94.1063%2C33.4268
wake village - nearest hotel: Country Inn & Suites, Texarkana, TX
City_ID              141
City              gebeit
Lat              21.0667
Lng              36.3167
Max Temp           24.85
Humidity            

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
87,87,wake village,33.4268,-94.1063,23.36,60,Clear,2.55,US,1727913645,"Country Inn & Suites, Texarkana, TX"
141,141,gebeit,21.0667,36.3167,24.85,44,Clear,2.49,SD,1727913706,No hotel found
162,162,ha'il,27.5219,41.6907,25.01,31,Clear,1.03,SA,1727913729,فندق كلاسيك
170,170,belmonte,-15.8631,-38.8828,23.04,82,Clear,4.16,BR,1727913740,No hotel found
173,173,sabha,27.0377,14.4283,24.85,30,Clear,3.25,LY,1727913743,فندق الكاظم-سبها
304,304,fort bragg,35.1390,-79.0060,22.91,88,Clear,2.06,US,1727913903,Airborne Inn Lodging
361,361,rawah,34.4816,41.9093,23.86,39,Clear,2.74,IQ,1727913972,No hotel found
375,375,siwa oasis,29.2041,25.5195,23.12,54,Clear,3.91,EG,1727913989,Flower of the city hotel
388,388,pakenham,-38.0667,145.4833,20.89,64,Clear,1.48,AU,1727914003,Pink Hill Hotel
445,445,lapithos,35.3375,33.1792,23.00,63,Clear,2.00,CY,1727914291,No hotel found


In [68]:
name_address

{'type': 'FeatureCollection', 'features': []}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [73]:
%%capture --no-display

# Configure the map plot
map_plot=hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles=True, 
    frame_width=800,
    frome_height=600, 
    size=10, 
    color='City', 
    hover_cols=['Hotel Name', 'Country']
    
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

In [74]:
!pwd

/Users/julia/repository/python-api-challenge
